In [1]:
import os
import xml.etree.ElementTree as ET
import html
import numpy as np
from math import log
from collections import Counter
import math
import heapq
from collections import defaultdict

from whoosh.fields import Schema, TEXT, ID, NUMERIC, BOOLEAN
from whoosh.index import create_in, open_dir
from whoosh.analysis import StandardAnalyzer
from whoosh.qparser import QueryParser
from whoosh.qparser import MultifieldParser
from whoosh.scoring import BM25F, WeightingModel, WeightLengthScorer
from whoosh.searching import Searcher
from whoosh.qparser import OrGroup

from sentence_transformers import SentenceTransformer, util

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

/Users/Paola/ir_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Initial Index
# === Inputs ===
inputs = [
    "/Users/Paola/Desktop/IR/softwareengineering.stackexchange.com",
    "/Users/Paola/Desktop/IR/cseducators.stackexchange.com/Posts.xml",
    "/Users/Paola/Desktop/IR/cstheory.stackexchange.com/Posts.xml",
    "/Users/Paola/Desktop/IR/cs.stackexchange.com/Posts.xml",
    "/Users/Paola/Desktop/IR/ai.stackexchange.com/Posts.xml",
    "/Users/Paola/Desktop/IR/datascience.stackexchange.com/Posts.xml"
]

# === Resolve valid XML files ===
xml_files = []
for path in inputs:
    if os.path.isdir(path):
        candidate = os.path.join(path, "Posts.xml")
        if os.path.exists(candidate):
            xml_files.append(candidate)
    elif path.lower().endswith(".xml") and os.path.exists(path):
        xml_files.append(path)

print("Indexing from these XML files:")
for f in xml_files:
    print(" ", f)

# === HTML tag stripper ===
TAG_RE = re.compile(r'<[^>]+>')
def clean_html(raw_html):
    return TAG_RE.sub('', html.unescape(raw_html))

# === Document aggregator ===
aggregated_docs = {}
for xml_file in xml_files:
    site_name = os.path.basename(os.path.dirname(xml_file)) if xml_file.endswith("Posts.xml") else os.path.basename(xml_file).replace(".xml", "")
    print(f"Reading posts from {site_name}...")
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for row in root.findall('row'):
        pid = row.attrib.get('Id')
        if not pid:
            continue
        post_type = row.attrib.get('PostTypeId')
        unique_id = f"{site_name}:{pid}"
        if post_type == "1":  # Question
            title = clean_html(row.attrib.get('Title', ""))
            body = clean_html(row.attrib.get('Body', ""))
            raw_tags = row.attrib.get('Tags', "")
            tag_list = raw_tags.strip("|").split("|") if raw_tags else []
            tags = ",".join(tag_list)
            aggregated_docs[unique_id] = {
                'title': title,
                'body': body,
                'tags': tags,
                'site': site_name,
                'original_id': pid
            }
        elif post_type == "2":  # Answer
            parent_pid = row.attrib.get('ParentId')
            if parent_pid:
                parent_uid = f"{site_name}:{parent_pid}"
                answer_body = clean_html(row.attrib.get('Body', ""))
                if parent_uid in aggregated_docs:
                    aggregated_docs[parent_uid]['body'] += "\n\n[Answer]: " + answer_body

# === Create Whoosh index ===
schema = Schema(
    post_id=ID(stored=True, unique=True),
    title=TEXT(stored=True, analyzer=StandardAnalyzer()),
    body=TEXT(stored=True, analyzer=StandardAnalyzer()),
    tags=TEXT(stored=True, analyzer=StandardAnalyzer())
)

if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = create_in("indexdir", schema)
writer = ix.writer()
print(f"Adding {len(aggregated_docs)} documents to the index…")
for qid, doc in aggregated_docs.items():
    writer.add_document(
        post_id=qid,
        title=doc['title'],
        body=doc['body'],
        tags=doc['tags']
    )
writer.commit()
print("Indexing complete!")


In [ ]:
# BM25
def tokenize_query(query_str):
    analyzer = StandardAnalyzer()
    return [token.text for token in analyzer(query_str)]
 
def search_with_bm25(query_text, title_boost=1.0, top_k=5):
    """
    Search using BM25 with field boosting
    """
    ix = open_dir("indexdir")
    
    # Tokenize query
    tokens = tokenize_query(query_text)
    print("Tokenized query (BM25):", tokens)
    query_text = " ".join(tokens)  # Replace original query with tokenized version

    # Configure BM25F with field boosting
    bm25 = BM25F(B=0.75, title_B=0.5, body_B=0.5, title_boost=0.5, body_boost=0.5)
    
    # Create parser
    parser = MultifieldParser(["title", "body"], ix.schema, group=OrGroup)
    
    # Parse and search
    q = parser.parse(query_text)
    with ix.searcher(weighting=bm25) as searcher:
        results = searcher.search(q, limit=top_k)
        print(f"\n=== BM25 Search Results (title_boost={title_boost}) ===")
        for i, hit in enumerate(results):
            print(f"{i+1}. [{hit['post_id']}] {hit['title']} (Score: {hit.score:.4f})")

# === Run search with example query ===
if __name__ == "__main__":
    query = "How do I detect a cycle in a graph?"

    # Search with BM25
    search_with_bm25(query)

In [12]:
# QLM
def tokenize(text):
    analyzer = StandardAnalyzer()
    return [token.text for token in analyzer(text)]

def compute_dirichlet_scores_for_field(searcher, query_terms, field, mu, weight):
    total_terms = searcher.field_length(field)
    doc_scores = defaultdict(float)
    doc_lengths = {
        docnum: searcher.doc_field_length(docnum, field)
        for docnum in range(searcher.doc_count_all())
    }

    # Compute P(q_i | C_f)
    term_collection_probs = {}
    for term in query_terms:
        cf = searcher.frequency(field, term)
        if cf > 0:
            term_collection_probs[term] = cf / total_terms

    for term, p_coll in term_collection_probs.items():
        matcher = searcher.postings(field, term)
        seen_docs = set()

        if matcher.is_active():
            while matcher.is_active():
                docnum = matcher.id()
                tf = matcher.value_as("frequency")
                doc_len = doc_lengths.get(docnum, 0) or 0
                score = math.log((tf + mu * p_coll) / (doc_len + mu))
                doc_scores[docnum] += weight * score
                seen_docs.add(docnum)
                matcher.next()

        for docnum in doc_lengths.keys():
            if docnum not in seen_docs:
                doc_len = doc_lengths.get(docnum, 0) or 0
                score = math.log((mu * p_coll) / (doc_len + mu))
                doc_scores[docnum] += weight * score

    return doc_scores

def query_likelihood_with_dirichlet_multifield(query, k=10, mu=2000, field_weights={"body": 0.5, "title": 0.8}):
    ix = open_dir("indexdir")
    query_terms = tokenize(query)
    combined_scores = defaultdict(float)

    with ix.searcher() as searcher:
        for field, weight in field_weights.items():
            field_scores = compute_dirichlet_scores_for_field(searcher, query_terms, field, mu, weight)
            for docnum, score in field_scores.items():
                combined_scores[docnum] += score

        top_k = heapq.nlargest(k, combined_scores.items(), key=lambda x: x[1])
        return [(searcher.stored_fields(docnum), score) for docnum, score in top_k]

results = query_likelihood_with_dirichlet_multifield("How to load a large json file in python?", k=5, mu=2000, field_weights={"body": 0.3, "title": 0.7})
for i, (doc, score) in enumerate(results):
    print(f"{i+1}. {doc['post_id']} — {doc['title']} — Score: {score:.4f}")


In [15]:
# Embedding reranking

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def rerank_with_embeddings(query, candidates, top_k=5):
    query_emb = embed_model.encode(query, convert_to_tensor=True)
    doc_texts = [doc["title"] + " " + doc["body"] for doc, _ in candidates]
    doc_embs = embed_model.encode(doc_texts, convert_to_tensor=True)

    similarities = util.cos_sim(query_emb, doc_embs)[0]
    scored = list(zip(candidates, similarities))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [(doc, score.item()) for (doc, _), score in scored[:top_k]]
    
query = "How do I detect a cycle in a graph?"
qlm_results = query_likelihood_with_dirichlet_multifield(query, k=20, mu=2000, field_weights={"body": 0.3, "title": 0.7})
reranked = rerank_with_embeddings(query, qlm_results, top_k=5)

print("=== Embedding-based Reranked Results ===")
for i, (doc, score) in enumerate(reranked):
    print(f"{i+1}. {doc['post_id']} — {doc['title']} — CosSim: {score:.4f}")
    # print(doc['body'])
    print(doc['post_id'])


/Users/Paola/ir_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/Paola/ir_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== Embedding-based Reranked Results ===
1. cs.stackexchange.com:63255 — Output cycle found by DFS — CosSim: 0.6395
cs.stackexchange.com:63255
2. cs.stackexchange.com:96918 — Detect non existence of a cycle in a graph using Datalog : SMTLIB Format for Z3 — CosSim: 0.5682
cs.stackexchange.com:96918
3. cstheory.stackexchange.com:49017 — Detect if a graph has a $k$ cycle in space complexity $O((\log k)^d)$ for fixed $d \geq1$ — CosSim: 0.5437
cstheory.stackexchange.com:49017
4. cs.stackexchange.com:75952 — detecting a cycle in an undirected graph problem is in $RL$ complexity class — CosSim: 0.5363
cs.stackexchange.com:75952
5. cs.stackexchange.com:75341 — Proving $\#CYCLE \in \#P$ — CosSim: 0.5202
cs.stackexchange.com:75341


In [230]:
# Pseudo relevance feedback

def relevance_model_rerank(query_str, fields=["body"], top_k_candidate=10, top_k_final=3, mu=2000):
    # Get top-k results with your QLM model
    initial_results = query_likelihood_with_dirichlet_multifield(
        query=query_str,
        k=top_k_candidate,
        mu=mu,
        field_weights={"body": 0.3, "title": 0.7}
    )

    # Separate docs and scores
    c_docs = [doc for doc, _ in initial_results]
    c_scores = {doc['post_id']: score for doc, score in initial_results}

    # Tokenize and collect term stats
    analyzer = StandardAnalyzer()
    stats = {}
    vocab = set()

    for doc in c_docs:
        combined_text = " ".join(doc.get(f, "") for f in fields)
        toks = [t.text for t in analyzer(combined_text)]
        cnts = Counter(toks)
        total = sum(cnts.values())
        stats[doc['post_id']] = {'cnts': cnts, 'tot': total}
        vocab.update(cnts)

    V = len(vocab)

    def p_w_D(ds, w):
        return (ds['cnts'].get(w, 0) + 1) / (ds['tot'] + V)

    # Build relevance model
    W = {}
    S = sum(c_scores.values())

    for w in vocab:
        W[w] = sum(c_scores[d['post_id']] * p_w_D(stats[d['post_id']], w) for d in c_docs) / (S or 1)

    # Rerank using RM1
    rerank = {}
    for doc in c_docs:
        score = sum(W[w] * log(p_w_D(stats[doc['post_id']], w)) for w in vocab)
        rerank[doc['post_id']] = score

    sorted_docs = sorted(c_docs, key=lambda d: rerank[d['post_id']], reverse=True)

    # Print and return top-k
    print(f"----- Top {top_k_final} Results Using RM Re-ranking -----")
    for i, doc in enumerate(sorted_docs[:top_k_final], 1):
        print(f"{i}. Post ID: {doc['post_id']}, Title: {doc.get('title', '')}")
        print(f"   Score: {rerank[doc['post_id']]:.4f}")
        print(f"   Body: {doc.get('body', '')}")
        # print(doc['post_id'])

    return sorted_docs[:top_k_final]

relevance_model_rerank(
    query_str="What’s the difference between Big-O, Big-Theta, and Big-Omega?",
    fields=["body", "title"],
    top_k_candidate=5,
    top_k_final=5,
    mu=2000
)

In [6]:
# Index with meta data

# === Inputs ===
input_dir = "/Users/Paola/Desktop/IR"
site_dirs = [
    "softwareengineering.stackexchange.com",
    "cseducators.stackexchange.com",
    "cstheory.stackexchange.com",
    "cs.stackexchange.com",
    "ai.stackexchange.com",
    "datascience.stackexchange.com"
]

# === HTML tag stripper ===
TAG_RE = re.compile(r'<[^>]+>')
def clean_html(raw_html):
    return TAG_RE.sub('', html.unescape(raw_html))

# === Read accepted answer IDs for each question ===
accepted_answers = {}
print("Scanning for accepted answers...")
for site in site_dirs:
    path = os.path.join(input_dir, site, "Posts.xml")
    tree = ET.parse(path)
    root = tree.getroot()
    for row in root.findall('row'):
        if row.attrib.get('PostTypeId') == "1":  # Question
            aid = row.attrib.get('AcceptedAnswerId')
            if aid:
                accepted_answers[f"{site}:{aid}"] = True

# === Document aggregator with filters and metadata ===
aggregated_docs = {}
print("Indexing filtered and boosted questions only...")
for site in site_dirs:
    path = os.path.join(input_dir, site, "Posts.xml")
    tree = ET.parse(path)
    root = tree.getroot()
    for row in root.findall('row'):
        pid = row.attrib.get('Id')
        if not pid:
            continue
        post_type = row.attrib.get('PostTypeId')
        unique_id = f"{site}:{pid}"

        if post_type == "1":  # Question
            answer_count = int(row.attrib.get("AnswerCount", "0"))
            if answer_count == 0:
                continue  # Filter: skip questions with no answers

            title = clean_html(row.attrib.get('Title', ""))
            body = clean_html(row.attrib.get('Body', ""))
            raw_tags = row.attrib.get('Tags', "")
            tag_list = raw_tags.strip("|").split("|") if raw_tags else []
            tags = ",".join(tag_list)
            score = int(row.attrib.get("Score", "0"))
            has_accepted = (
                row.attrib.get("AcceptedAnswerId") 
                and f"{site}:{row.attrib['AcceptedAnswerId']}" in accepted_answers
            )

            aggregated_docs[unique_id] = {
                'title': title,
                'body': body,
                'tags': tags,
                'score': score,
                'answer_count': answer_count,
                'has_accepted': has_accepted,
            }
        elif post_type == "2":  # Answer
            parent_pid = row.attrib.get('ParentId')
            if parent_pid:
                parent_uid = f"{site}:{parent_pid}"
                answer_body = clean_html(row.attrib.get('Body', ""))
                if parent_uid in aggregated_docs:
                    aggregated_docs[parent_uid]['body'] += "\n\n[Answer]: " + answer_body

# === Create Whoosh index with metadata ===
schema = Schema(
    post_id=ID(stored=True, unique=True),
    title=TEXT(stored=True, analyzer=StandardAnalyzer()),
    body=TEXT(stored=True, analyzer=StandardAnalyzer()),
    tags=TEXT(stored=True, analyzer=StandardAnalyzer()),
    score=NUMERIC(stored=True),
    answer_count=NUMERIC(stored=True),
    has_accepted=BOOLEAN(stored=True)
)

if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = create_in("indexdir", schema)
writer = ix.writer()
print(f"Adding {len(aggregated_docs)} documents to the index…")
for qid, doc in aggregated_docs.items():
    writer.add_document(
        post_id=qid,
        title=doc['title'],
        body=doc['body'],
        tags=doc['tags'],
        score=doc['score'],
        answer_count=doc['answer_count'],
        has_accepted=doc['has_accepted']
    )
writer.commit()
print("Indexing complete!")

Scanning for accepted answers...
Indexing filtered and boosted questions only...
Adding 145818 documents to the index…
Indexing complete!


In [13]:
# QLM with metadata
def query_likelihood_with_dirichlet_multifield(query, k=10, mu=2000, field_weights={"body": 0.5, "title": 0.8},
                                               accepted_boost=1.1, answer_boost_scale=0.05):
    ix = open_dir("indexdir")
    query_terms = tokenize(query)
    combined_scores = defaultdict(float)

    with ix.searcher() as searcher:
        for field, weight in field_weights.items():
            field_scores = compute_dirichlet_scores_for_field(searcher, query_terms, field, mu, weight)
            for docnum, score in field_scores.items():
                combined_scores[docnum] += score

        # Apply boosts
        boosted_scores = {}
        for docnum, score in combined_scores.items():
            doc = searcher.stored_fields(docnum)
            boost = 1.0
            if doc.get("has_accepted") == "yes":
                boost *= accepted_boost
            answer_count = int(doc.get("answer_count", 0))
            boost *= (1 + answer_boost_scale * answer_count)
            boosted_scores[docnum] = score * boost

        top_k = heapq.nlargest(k, boosted_scores.items(), key=lambda x: x[1])
        return [(searcher.stored_fields(docnum), score) for docnum, score in top_k]

query = "How do I efficiently load a large json file in Python?"
qlm_results = query_likelihood_with_dirichlet_multifield(
    query,
    k=20,
    mu=2000,
    field_weights={"body": 0.3, "title": 0.7},
    accepted_boost=1.15,
    answer_boost_scale=0.03
)
reranked = rerank_with_embeddings(query, qlm_results, top_k=10)

for i, (doc, score) in enumerate(reranked):
    print(f"{i+1}. {doc['post_id']} — {doc['title']} — CosSim: {score:.4f}")

In [33]:
# Pseudo ranking SVM
def pseudo_relevance_training(queries, k=6, mu=2000, field_weights={"body": 0.3, "title": 0.7},
                               accepted_boost=1.15, answer_boost_scale=0.03,
                               return_docs=False):
    X, y = [], []
    docs_out = []  # to store documents if return_docs is True

    for query in queries:
        top_docs = query_likelihood_with_dirichlet_multifield(
            query=query,
            k=k,
            mu=mu,
            field_weights=field_weights,
            accepted_boost=accepted_boost,
            answer_boost_scale=answer_boost_scale
        )

        midpoint = len(top_docs) // 2
        pseudo_labels = [1 if i < midpoint else 0 for i in range(len(top_docs))]

        query_emb = embed_model.encode(query, convert_to_tensor=True)

        for i, ((doc, qlm_score), label) in enumerate(zip(top_docs, pseudo_labels)):
            doc_text = doc["title"] + " " + doc["body"]
            doc_emb = embed_model.encode(doc_text, convert_to_tensor=True)
            emb_sim = util.cos_sim(query_emb, doc_emb).item()

            meta_boost = (
                (1.0 if doc.get("has_accepted") else 0.0)
                + log(1 + doc.get("answer_count", 0)) * answer_boost_scale
            )

            X.append([qlm_score, emb_sim, meta_boost])
            y.append(label)
            if return_docs:
                docs_out.append(doc)

    if return_docs:
        return np.array(X), docs_out
    else:
        return np.array(X), np.array(y)


In [ ]:
queries = [
    "How do I efficiently load a large json file in Python?",
    "What’s the difference between breadth-first search and depth-first search?",
    "What are hash maps and how do they work?",
    "How does memoization improve recursive algorithms?",
    "How does garbage collection work in Java?",
    "What’s the difference between static and dynamic typing?",
    "What is a memory leak and how can I detect it?",
    "How does Python manage memory?",
    "When should I use a stack over a queue?",
    "How does a binary search algorithm work?",
    "How do I resolve circular import errors in Python?",
    "What is the difference between process and thread?",
    "How does a priority queue work internally?",
    "What is the best way to handle exceptions in Python?",
    "What is the difference between synchronous and asynchronous programming?",
    "How do I serialize and deserialize a Python object?",
    "What’s the difference between REST and GraphQL?",
    "What is tail recursion and why is it useful?",
    "How do I avoid race conditions in multithreading?",
    "What are the advantages of functional programming?",
    "How does version control with Git work?",
    "What is a deadlock and how can it be prevented?",
    "How does a Trie data structure work?",
    "What is the difference between TCP and UDP?",
    "How does dynamic programming differ from greedy algorithms?",
    "What are callbacks and how are they used in JavaScript?",
    "How can I optimize database queries for performance?",
    "What is a state machine and where is it used?",
    "How do I implement a stack using queues?",
    "What are lambda functions in Python?",
    "What is the difference between shallow and deep copy in Python?",
    "How do I handle large datasets in pandas?",
    "What is the difference between compile-time and runtime errors?",
    "How can I improve the time complexity of my code?",
    "What is object-oriented programming and how does it differ from procedural?",
    "What are the differences between interfaces and abstract classes in Java?",
    "How does inheritance work in Python?",
    "How does the call stack work in recursion?",
    "What is the purpose of the virtual keyword in C++?",
    "What is dependency injection and why is it used?",
    "How do I implement caching in Python for faster lookups?",
    "How do I implement a queue using two stacks?",
    "What is the difference between synchronous and asynchronous calls?",
    "How can I optimize a SQL query that uses multiple joins?",
    "How do I detect memory leaks in a Java application?",
    "How do I merge two sorted arrays in C++?",
    "Why does using global variables cause issues in multithreaded programs?",
    "How do I convert a JSON file into a Python dictionary?",
    "What are the trade-offs between depth-first and breadth-first traversal?",
    "How do I fix a segmentation fault in C?",
    "What causes a stack overflow error in recursive functions?",
    "How does memoization improve performance?",
    "Why is quicksort faster than bubble sort in practice?",
    "How can I write a tail-recursive function in Python?",
    "What’s the time complexity of inserting into a binary search tree?"
]

X, y = pseudo_relevance_training(queries)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear', probability=True))
])
model.fit(X, y)


In [ ]:
# Your test queries
test_queries = [
    "How do I detect a cycle in a graph?",
]

# Run pseudo relevance feature extraction
X_test, docs_test = pseudo_relevance_training(test_queries, return_docs=True)

# Predict scores for each doc using your trained model
scores = model.decision_function(X_test)  # shape = (num_docs,)

# Group scores and docs per query
docs_per_query = 5
for i, query in enumerate(test_queries):
    start = i * docs_per_query
    end = start + docs_per_query

    query_scores = scores[start:end]
    query_docs = docs_test[start:end]

    # Sort by score
    ranked = sorted(zip(query_docs, query_scores), key=lambda x: x[1], reverse=True)[:5]

    print(f"\nQuery: {query}")
    print("Top 5 ranked documents:")
    for rank, (doc, score) in enumerate(ranked, 1):
        print(f"{rank}. {doc['post_id']} — {doc['title']} — Score: {score:.4f}")
        # print(doc['body'])